In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = \
  '--conf spark.cassandra.connection.host=cassandra --packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.2,com.datastax.spark:spark-cassandra-connector_2.11:2.0.2 pyspark-shell'

In [2]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

In [3]:
sc = SparkContext(appName="BigDataRiver")
sc.setLogLevel("WARN")
sc.setCheckpointDir('checkpoint/')
ssc = StreamingContext(sc, 60)
sql = SQLContext(sc)

In [4]:
kafkaStream = KafkaUtils.createDirectStream(ssc, ['bdr'], {"metadata.broker.list": 'kafka:9092'})

In [5]:
parsed = kafkaStream.map(lambda v: v[1])

In [6]:
#split is_purchase column into two
separateClicksSchema = StructType([   
    StructField("purchased_count", LongType(), False),
    StructField("clicked_count", LongType(), False)
])

def separateClicks(is_purchase):
  return (is_purchase, 1-is_purchase)

separateClicks_udf = F.udf(separateClicks, separateClicksSchema)

In [7]:
def buildCFModel(train):
    def isProductToRating(productCount, clickCount):
        return (productCount * 3.0) + clickCount
    
    ratings = train.rdd.\
        map(lambda r: Rating(r.user_id, r.product, isProductToRating(r.purchased_count, r.clicked_count)))
    rank = 10
    numIterations = 20
    lambdaFactor = 0.01
    alpha = 0.01
    seed = 42
    return ALS.trainImplicit(ratings, rank, numIterations, alpha, seed=seed)

In [8]:
def recommendTopProducts(dfModel):
        numberOfRecommendationsRequired = 5
        rdd = dfModel.recommendProductsForUsers(numberOfRecommendationsRequired)
        recommendations = rdd.map(lambda (user,ratings): (user, map(lambda r: r.product, ratings)))
        topRecommendationsSchema = StructType([
            StructField("user_id", IntegerType(), False),
            StructField("recommended_products", ArrayType(IntegerType()), False)
        ])
        return sql.createDataFrame(recommendations, topRecommendationsSchema)

In [9]:
def processStream(rdd):
    df = sql.read.json(rdd)
    if(len(df.columns)):
        #store updated counters in C*
        df.withColumn('c', separateClicks_udf(df['is_purchase'])).\
            select("user_id","product","c.purchased_count","c.clicked_count").\
            write.format("org.apache.spark.sql.cassandra").mode('append').\
            options(table="users_interests", keyspace="bdr").save()
            
        #read all data from C*
        usersInterests = sql.read.format("org.apache.spark.sql.cassandra").\
            options(table="users_interests", keyspace="bdr").load().cache()

        dfModel = buildCFModel(usersInterests.select("user_id","product","clicked_count","purchased_count"))
        top5 = recommendTopProducts(dfModel)
        top5.show()
        top5.write.format("org.apache.spark.sql.cassandra").mode('append').options(table="cf", keyspace="bdr").save()
            
        print "Saved"
    else:
        print "Empty"

In [ ]:
parsed.foreachRDD(lambda rdd: processStream(rdd))

In [ ]:
ssc.start()
ssc.awaitTermination()

+-------+--------------------+
|user_id|recommended_products|
+-------+--------------------+
|   1125| [9, 36, 29, 64, 67]|
|   3800|  [9, 29, 36, 67, 5]|
|   1825| [99, 29, 40, 6, 17]|
|   5625|[46, 42, 18, 29, 64]|
|   9626|[81, 78, 85, 55, 77]|
|   3651|[78, 55, 40, 29, 17]|
|   3751|  [82, 2, 99, 3, 62]|
|   3802|[21, 95, 93, 37, 98]|
|   1277|[42, 18, 91, 46, 50]|
|   2002| [99, 29, 40, 6, 17]|
|   4152|[95, 93, 21, 98, 37]|
|   4752|[62, 93, 57, 95, 21]|
|   1427|[11, 46, 68, 42, 18]|
|   1903|[46, 42, 18, 29, 11]|
|   3228| [5, 99, 36, 30, 37]|
|   7528| [42, 18, 46, 5, 29]|
|   3003|[62, 57, 11, 93, 46]|
|   2403| [81, 78, 85, 62, 3]|
|   5079| [37, 95, 21, 5, 98]|
|   9979| [42, 18, 46, 91, 5]|
+-------+--------------------+
only showing top 20 rows

Saved
+-------+--------------------+
|user_id|recommended_products|
+-------+--------------------+
|   8125|  [91, 10, 48, 9, 2]|
|   8500|  [9, 29, 36, 3, 64]|
|   3975| [30, 5, 62, 37, 15]|
|   1125|  [9, 3, 91, 21, 11]|
|   995